In [60]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
# подключаем библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import numpy.ma as ma
# from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
print ("Train data shape:", train.shape)
print ("Test data shape:", test.shape)

Train data shape: (1460, 81)
Test data shape: (1459, 80)


In [4]:
train.head(20
        )

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [5]:
# Процентный список пропущенных данных
# for col in train.columns:
#     pct_missing = np.mean(train[col].isnull())
#     print('{} - {}%'.format(col, round(pct_missing*100)))
    

In [6]:
# impute the missing values and create the missing value indicator variables for each numeric column.
train_numeric = train.select_dtypes(include=[np.number])
numeric_cols = train_numeric.columns.values

for col in numeric_cols:
    missing = train[col].isnull()
    num_missing = np.sum(missing)
    
    if num_missing > 0:  # only do the imputation for the columns that have missing values.
        print('imputing missing values for: {}'.format(col))
        train['{}_ismissing'.format(col)] = missing
        med = train[col].median()
        train[col] = train[col].fillna(med)

imputing missing values for: LotFrontage
imputing missing values for: MasVnrArea
imputing missing values for: GarageYrBlt


In [7]:
# Процентный список пропущенных данных
# for col in train.columns:
#     pct_missing = np.mean(train[col].isnull())
#     print('{} - {}%'.format(col, round(pct_missing*100)))
    

In [8]:
# train.info()

In [9]:
train.dtypes

Id                         int64
MSSubClass                 int64
MSZoning                  object
LotFrontage              float64
LotArea                    int64
                          ...   
SaleCondition             object
SalePrice                  int64
LotFrontage_ismissing       bool
MasVnrArea_ismissing        bool
GarageYrBlt_ismissing       bool
Length: 84, dtype: object

In [10]:
cols_to_drop = ['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_less = train.drop(cols_to_drop, axis=1, inplace=True)

In [11]:
# Процентный список пропущенных данных
for col in train.columns:
    pct_missing = np.mean(train[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))
    
    

Id - 0.0%
MSSubClass - 0.0%
MSZoning - 0.0%
LotFrontage - 0.0%
LotArea - 0.0%
Street - 0.0%
LotShape - 0.0%
LandContour - 0.0%
Utilities - 0.0%
LotConfig - 0.0%
LandSlope - 0.0%
Neighborhood - 0.0%
Condition1 - 0.0%
Condition2 - 0.0%
BldgType - 0.0%
HouseStyle - 0.0%
OverallQual - 0.0%
OverallCond - 0.0%
YearBuilt - 0.0%
YearRemodAdd - 0.0%
RoofStyle - 0.0%
RoofMatl - 0.0%
Exterior1st - 0.0%
Exterior2nd - 0.0%
MasVnrType - 1.0%
MasVnrArea - 0.0%
ExterQual - 0.0%
ExterCond - 0.0%
Foundation - 0.0%
BsmtQual - 3.0%
BsmtCond - 3.0%
BsmtExposure - 3.0%
BsmtFinType1 - 3.0%
BsmtFinSF1 - 0.0%
BsmtFinType2 - 3.0%
BsmtFinSF2 - 0.0%
BsmtUnfSF - 0.0%
TotalBsmtSF - 0.0%
Heating - 0.0%
HeatingQC - 0.0%
CentralAir - 0.0%
Electrical - 0.0%
1stFlrSF - 0.0%
2ndFlrSF - 0.0%
LowQualFinSF - 0.0%
GrLivArea - 0.0%
BsmtFullBath - 0.0%
BsmtHalfBath - 0.0%
FullBath - 0.0%
HalfBath - 0.0%
BedroomAbvGr - 0.0%
KitchenAbvGr - 0.0%
KitchenQual - 0.0%
TotRmsAbvGrd - 0.0%
Functional - 0.0%
Fireplaces - 0.0%
GarageType

In [12]:
# заполним пустые значения

In [13]:
#One-hot encoding

In [14]:
train= train.dropna()

In [15]:
categorical_columns = train.columns[train.dtypes == 'object']


In [16]:
onehot_encoder = OneHotEncoder(sparse=False)

encoded_categorical_columns = pd.DataFrame(onehot_encoder.fit_transform(train[categorical_columns]))
encoded_categorical_columns.head()

,0,1,2,3,4,5,6,7,8,9,...,219,220,221,222,223,224,225,226,227,228
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
# train_test_split

In [18]:
train.dtypes

Id                         int64
MSSubClass                 int64
MSZoning                  object
LotFrontage              float64
LotArea                    int64
                          ...   
SaleCondition             object
SalePrice                  int64
LotFrontage_ismissing       bool
MasVnrArea_ismissing        bool
GarageYrBlt_ismissing       bool
Length: 79, dtype: object

In [19]:
X = train.drop('SalePrice',axis=1)
y = train['SalePrice']

In [20]:
# print ("Train data shape:", X_train.shape)
# print ("Test data shape:", y_train.shape)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [22]:
# classifier = LogisticRegression(solver='lbfgs',random_state=0)

In [23]:
# classifier.fit(X_train, y_train)

In [24]:
# catboost

In [25]:
from catboost import CatBoostRegressor

In [26]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [29]:

model= CatBoostRegressor(iterations=2000,learning_rate=1,depth = 2)

In [30]:
model.fit(X,y,categorical_features_indices)

0:	learn: 54923.1891255	total: 24.1ms	remaining: 48.3s
1:	learn: 50200.0310838	total: 40.2ms	remaining: 40.1s
2:	learn: 48190.1234152	total: 55.3ms	remaining: 36.8s
3:	learn: 46826.3867360	total: 65.2ms	remaining: 32.6s
4:	learn: 44753.5363475	total: 75.6ms	remaining: 30.2s
5:	learn: 41710.8260293	total: 85.6ms	remaining: 28.5s
6:	learn: 40599.1931161	total: 99ms	remaining: 28.2s
7:	learn: 39578.3517703	total: 109ms	remaining: 27s
8:	learn: 39437.0729736	total: 119ms	remaining: 26.2s
9:	learn: 39264.0246825	total: 128ms	remaining: 25.5s
10:	learn: 38344.3455190	total: 138ms	remaining: 24.9s
11:	learn: 38074.9894979	total: 147ms	remaining: 24.4s
12:	learn: 36856.0497152	total: 157ms	remaining: 24s
13:	learn: 35878.7784602	total: 166ms	remaining: 23.6s
14:	learn: 34229.5654916	total: 176ms	remaining: 23.3s
15:	learn: 33848.0117964	total: 185ms	remaining: 23s
16:	learn: 33466.6856028	total: 195ms	remaining: 22.8s
17:	learn: 33175.9974194	total: 205ms	remaining: 22.6s
18:	learn: 33058.9790

158:	learn: 22873.0492006	total: 1.84s	remaining: 21.4s
159:	learn: 22856.8946198	total: 1.86s	remaining: 21.4s
160:	learn: 22840.5125703	total: 1.87s	remaining: 21.4s
161:	learn: 22834.3019240	total: 1.88s	remaining: 21.3s
162:	learn: 22810.7783236	total: 1.89s	remaining: 21.3s
163:	learn: 22772.8539324	total: 1.9s	remaining: 21.3s
164:	learn: 22759.8834215	total: 1.91s	remaining: 21.3s
165:	learn: 22748.9905374	total: 1.92s	remaining: 21.2s
166:	learn: 22727.1917822	total: 1.93s	remaining: 21.2s
167:	learn: 22713.1232958	total: 1.94s	remaining: 21.2s
168:	learn: 22690.7547896	total: 1.95s	remaining: 21.1s
169:	learn: 22612.6838001	total: 1.96s	remaining: 21.1s
170:	learn: 22564.1373749	total: 1.97s	remaining: 21.1s
171:	learn: 22550.8643083	total: 1.98s	remaining: 21.1s
172:	learn: 22547.1104751	total: 1.99s	remaining: 21s
173:	learn: 22522.2136494	total: 2s	remaining: 21s
174:	learn: 22509.5540380	total: 2.01s	remaining: 21s
175:	learn: 22490.9419994	total: 2.02s	remaining: 21s
176:

309:	learn: 19252.7594145	total: 3.49s	remaining: 19s
310:	learn: 19239.6722115	total: 3.5s	remaining: 19s
311:	learn: 19234.4820341	total: 3.52s	remaining: 19s
312:	learn: 19217.8696925	total: 3.53s	remaining: 19s
313:	learn: 19216.7664993	total: 3.54s	remaining: 19s
314:	learn: 19208.9385292	total: 3.55s	remaining: 19s
315:	learn: 19149.9910139	total: 3.56s	remaining: 19s
316:	learn: 19137.6964444	total: 3.57s	remaining: 18.9s
317:	learn: 19135.7005499	total: 3.58s	remaining: 18.9s
318:	learn: 19133.8435205	total: 3.59s	remaining: 18.9s
319:	learn: 19106.1301745	total: 3.6s	remaining: 18.9s
320:	learn: 19087.3716125	total: 3.61s	remaining: 18.9s
321:	learn: 19081.2057904	total: 3.62s	remaining: 18.8s
322:	learn: 19077.5425524	total: 3.63s	remaining: 18.8s
323:	learn: 19077.3789595	total: 3.64s	remaining: 18.8s
324:	learn: 19063.3424662	total: 3.65s	remaining: 18.8s
325:	learn: 19048.2885126	total: 3.65s	remaining: 18.8s
326:	learn: 19046.0898068	total: 3.67s	remaining: 18.8s
327:	lea

459:	learn: 17126.7494969	total: 5.12s	remaining: 17.1s
460:	learn: 17064.8350818	total: 5.13s	remaining: 17.1s
461:	learn: 17056.1359520	total: 5.14s	remaining: 17.1s
462:	learn: 17055.4056358	total: 5.15s	remaining: 17.1s
463:	learn: 17047.8811569	total: 5.16s	remaining: 17.1s
464:	learn: 17043.8871113	total: 5.17s	remaining: 17.1s
465:	learn: 17042.6728649	total: 5.18s	remaining: 17.1s
466:	learn: 17037.7727169	total: 5.19s	remaining: 17s
467:	learn: 17033.9873777	total: 5.2s	remaining: 17s
468:	learn: 17033.4477333	total: 5.21s	remaining: 17s
469:	learn: 17029.5770166	total: 5.22s	remaining: 17s
470:	learn: 17026.8432461	total: 5.24s	remaining: 17s
471:	learn: 17023.7517897	total: 5.25s	remaining: 17s
472:	learn: 17021.6945683	total: 5.26s	remaining: 17s
473:	learn: 17014.2119565	total: 5.27s	remaining: 17s
474:	learn: 17011.4237395	total: 5.28s	remaining: 17s
475:	learn: 16986.9420918	total: 5.3s	remaining: 17s
476:	learn: 16986.8225222	total: 5.32s	remaining: 17s
477:	learn: 1694

613:	learn: 15561.9339828	total: 7.12s	remaining: 16.1s
614:	learn: 15561.3026807	total: 7.13s	remaining: 16.1s
615:	learn: 15557.4717901	total: 7.15s	remaining: 16.1s
616:	learn: 15555.5447416	total: 7.16s	remaining: 16.1s
617:	learn: 15554.2417674	total: 7.17s	remaining: 16s
618:	learn: 15510.2388122	total: 7.19s	remaining: 16s
619:	learn: 15508.8562952	total: 7.2s	remaining: 16s
620:	learn: 15502.6629951	total: 7.22s	remaining: 16s
621:	learn: 15500.9480695	total: 7.24s	remaining: 16s
622:	learn: 15482.7272953	total: 7.25s	remaining: 16s
623:	learn: 15460.1632056	total: 7.27s	remaining: 16s
624:	learn: 15455.6752927	total: 7.28s	remaining: 16s
625:	learn: 15453.9692825	total: 7.29s	remaining: 16s
626:	learn: 15449.8331175	total: 7.3s	remaining: 16s
627:	learn: 15446.0847136	total: 7.32s	remaining: 16s
628:	learn: 15398.9582424	total: 7.33s	remaining: 16s
629:	learn: 15388.8494954	total: 7.35s	remaining: 16s
630:	learn: 15388.8480150	total: 7.37s	remaining: 16s
631:	learn: 15379.2942

...

1353:	learn: 10664.6760877	total: 16.9s	remaining: 8.07s
1354:	learn: 10664.4756374	total: 16.9s	remaining: 8.06s
1355:	learn: 10664.0793627	total: 16.9s	remaining: 8.04s
1356:	learn: 10661.6417697	total: 16.9s	remaining: 8.03s
1357:	learn: 10658.7257093	total: 17s	remaining: 8.02s
1358:	learn: 10654.1478309	total: 17s	remaining: 8s
1359:	learn: 10654.0870126	total: 17s	remaining: 7.99s
1360:	learn: 10651.4567889	total: 17s	remaining: 7.97s
1361:	learn: 10650.7282064	total: 17s	remaining: 7.96s
1362:	learn: 10650.4257904	total: 17s	remaining: 7.95s
1363:	learn: 10639.9528318	total: 17s	remaining: 7.93s
1364:	learn: 10639.3926864	total: 17s	remaining: 7.92s
1365:	learn: 10629.9562720	total: 17s	remaining: 7.91s
1366:	learn: 10594.0766949	total: 17s	remaining: 7.89s
1367:	learn: 10584.5376300	total: 17.1s	remaining: 7.88s
1368:	learn: 10582.3109644	total: 17.1s	remaining: 7.87s
1369:	learn: 10573.0901144	total: 17.1s	remaining: 7.85s
1370:	learn: 10569.2185845	total: 17.1s	remaining: 7.8

1498:	learn: 10026.3208826	total: 18.5s	remaining: 6.19s
1499:	learn: 10020.7174904	total: 18.5s	remaining: 6.18s
1500:	learn: 10003.7357296	total: 18.6s	remaining: 6.17s
1501:	learn: 9980.7236703	total: 18.6s	remaining: 6.16s
1502:	learn: 9978.3843208	total: 18.6s	remaining: 6.15s
1503:	learn: 9977.2788531	total: 18.6s	remaining: 6.13s
1504:	learn: 9973.3830647	total: 18.6s	remaining: 6.12s
1505:	learn: 9967.8608182	total: 18.6s	remaining: 6.11s
1506:	learn: 9957.4903175	total: 18.6s	remaining: 6.1s
1507:	learn: 9946.8296582	total: 18.7s	remaining: 6.09s
1508:	learn: 9936.0053170	total: 18.7s	remaining: 6.07s
1509:	learn: 9933.9034723	total: 18.7s	remaining: 6.06s
1510:	learn: 9931.1658658	total: 18.7s	remaining: 6.05s
1511:	learn: 9927.7606338	total: 18.7s	remaining: 6.04s
1512:	learn: 9927.1800114	total: 18.7s	remaining: 6.02s
1513:	learn: 9926.5109169	total: 18.7s	remaining: 6.01s
1514:	learn: 9925.7371573	total: 18.7s	remaining: 6s
1515:	learn: 9925.4478830	total: 18.8s	remaining:

1646:	learn: 9299.1135569	total: 20.5s	remaining: 4.4s
1647:	learn: 9297.7398650	total: 20.5s	remaining: 4.39s
1648:	learn: 9285.8927225	total: 20.6s	remaining: 4.38s
1649:	learn: 9276.1558176	total: 20.6s	remaining: 4.36s
1650:	learn: 9263.5725428	total: 20.6s	remaining: 4.35s
1651:	learn: 9254.7662101	total: 20.6s	remaining: 4.34s
1652:	learn: 9248.2203973	total: 20.6s	remaining: 4.33s
1653:	learn: 9245.8084880	total: 20.6s	remaining: 4.31s
1654:	learn: 9243.3402247	total: 20.6s	remaining: 4.3s
1655:	learn: 9241.7121991	total: 20.7s	remaining: 4.29s
1656:	learn: 9241.4468970	total: 20.7s	remaining: 4.28s
1657:	learn: 9238.1257567	total: 20.7s	remaining: 4.26s
1658:	learn: 9237.1727094	total: 20.7s	remaining: 4.25s
1659:	learn: 9234.9793930	total: 20.7s	remaining: 4.24s
1660:	learn: 9232.6870091	total: 20.7s	remaining: 4.23s
1661:	learn: 9230.8703867	total: 20.7s	remaining: 4.22s
1662:	learn: 9227.8953348	total: 20.8s	remaining: 4.2s
1663:	learn: 9225.9221840	total: 20.8s	remaining: 4

1797:	learn: 8745.8679534	total: 22.5s	remaining: 2.53s
1798:	learn: 8715.6886007	total: 22.5s	remaining: 2.52s
1799:	learn: 8713.4543500	total: 22.5s	remaining: 2.5s
1800:	learn: 8711.2841664	total: 22.6s	remaining: 2.49s
1801:	learn: 8708.0751538	total: 22.6s	remaining: 2.48s
1802:	learn: 8707.9591445	total: 22.6s	remaining: 2.47s
1803:	learn: 8705.6783976	total: 22.6s	remaining: 2.46s
1804:	learn: 8705.2432330	total: 22.6s	remaining: 2.44s
1805:	learn: 8698.5175534	total: 22.6s	remaining: 2.43s
1806:	learn: 8691.3042852	total: 22.6s	remaining: 2.42s
1807:	learn: 8689.0016493	total: 22.7s	remaining: 2.41s
1808:	learn: 8686.5793367	total: 22.7s	remaining: 2.39s
1809:	learn: 8680.4802390	total: 22.7s	remaining: 2.38s
1810:	learn: 8678.3268842	total: 22.7s	remaining: 2.37s
1811:	learn: 8676.7486302	total: 22.7s	remaining: 2.36s
1812:	learn: 8674.3462770	total: 22.7s	remaining: 2.35s
1813:	learn: 8670.8891363	total: 22.7s	remaining: 2.33s
1814:	learn: 8666.9491516	total: 22.8s	remaining:

1945:	learn: 8240.4088863	total: 24.5s	remaining: 680ms
1946:	learn: 8239.9726084	total: 24.5s	remaining: 668ms
1947:	learn: 8238.3302110	total: 24.5s	remaining: 655ms
1948:	learn: 8235.6636600	total: 24.6s	remaining: 643ms
1949:	learn: 8232.4550199	total: 24.6s	remaining: 630ms
1950:	learn: 8227.0361292	total: 24.6s	remaining: 618ms
1951:	learn: 8218.4538921	total: 24.6s	remaining: 605ms
1952:	learn: 8216.6456121	total: 24.6s	remaining: 592ms
1953:	learn: 8216.1646867	total: 24.6s	remaining: 580ms
1954:	learn: 8214.7345032	total: 24.6s	remaining: 567ms
1955:	learn: 8204.8498492	total: 24.7s	remaining: 555ms
1956:	learn: 8201.9110225	total: 24.7s	remaining: 542ms
1957:	learn: 8201.2522792	total: 24.7s	remaining: 529ms
1958:	learn: 8196.9600239	total: 24.7s	remaining: 517ms
1959:	learn: 8196.9197367	total: 24.7s	remaining: 504ms
1960:	learn: 8194.8446489	total: 24.7s	remaining: 492ms
1961:	learn: 8187.4055193	total: 24.7s	remaining: 479ms
1962:	learn: 8183.1118724	total: 24.7s	remaining